## Wysokopoziomowy interfejs Keras

### Importy

In [ ]:
import matplotlib.pyplot as plt

import tensorflow as tf
print(f"TensorFlow version: {tf.__version__}")

In [ ]:
# Załadowanie rozszerzenia tensorboard
%load_ext tensorboard

In [ ]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

print(f"Dane treningowe: {len(x_train)} próbek")
print(f"Dane testowe: {len(x_test)} próbek")

### Budowanie modelu

Keras posiada dwie metody tworzenia modelu sieci neuronowej:
* sekwencyjne API (aka model sekwencyjny - tf.keras.models.Sequential)
* funkcyjne API

Najpierw zdefiniujmy model przy użyciu bardziej popularnego, **sekwencyjnego api**.

In [ ]:
model = tf.keras.models.Sequential()

layer_1 = tf.keras.layers.Flatten(input_shape=(28, 28))
layer_2 = tf.keras.layers.Dense(128, activation='relu')
layer_3 = tf.keras.layers.Dropout(0.2)  # regularization layer
layer_4 = tf.keras.layers.Dense(10, activation='softmax')  # normalization layer

model.add(layer_1)
model.add(layer_2)
model.add(layer_3)
model.add(layer_4)


# # moglibyśmy to równie dobrze wszystko zmieścić w jednym wywołaniu
# model = tf.keras.models.Sequential([
#     tf.keras.layers.Flatten(input_shape=(28, 28)),
#     tf.keras.layers.Dense(128, activation='relu'),
#     tf.keras.layers.Dropout(0.2),
#     tf.keras.layers.Dense(10, activation='softmax')
# ])

A jak wyglądałoby definiowanie modelu przy użyciu funkcyjnego API ?

In [ ]:
# warstwę poprzednią przekazujemy do warstwy następującej po niej
input_layer = tf.keras.layers.Input(shape=(28, 28))
layer_1 = tf.keras.layers.Flatten(input_shape=(28, 28))(input_layer)
layer_2 = tf.keras.layers.Dense(128, activation='relu')(layer_1)
layer_3 = tf.keras.layers.Dropout(0.2)(layer_2)
output_layer = tf.keras.layers.Dense(10, activation='softmax')(layer_3)

# i definiujemy model przekazując mu warstwę wejściową oraz warstwę wyjściową
model = tf.keras.models.Model(inputs=input_layer, outputs=output_layer)

Dla każdego przykładu model zwraca wektor wyników [logit](https://developers.google.com/machine-learning/glossary#logits) lub [log-odds](https://developers.google.com/machine-learning/glossary#log-odds) , po jednym dla każdej klasy.

In [ ]:
predictions = model(x_train[:1]).numpy()
predictions

### Konfiguracja i kompilacja modelu

In [ ]:
model.compile(
    optimizer='adam',  # optymalizator adam/SGD
    loss='sparse_categorical_crossentropy',  # funkcja strat 
    metrics=['accuracy']  # metryki
)

**adam** (ang. **ada**ptive **m**oment estimation) - optymalizator, można porównać z dodawaniem lub zmniejszaniem przyśpieszenia w zależności od stromizny zbocza podczas poruszania się po pagórkowatym terenie.

**sparse_categorical_crossentropy** to funkcja strat równa ujemnemu logarytmicznemu prawdopodobieństwu prawdziwej klasy. Jej wartość wynosi zero, jeśli model jest pewien, że klasa jest prawidłowa.

**accuracy** - dokładność modelu (stosunek poprawnych predykcji do całkowitej liczby predykcji)

In [ ]:
# Tworzenie callbacku na potrzeby tensorboard
import datetime

log_dir = 'adam_' + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

### Trenowanie modelu razem z walidacją (sprawdzeniem wydajności modelu)

In [ ]:
r = model.fit(
    x_train, 
    y_train, 
    validation_data=(x_test, y_test), 
    epochs=5,
    callbacks=[
        tensorboard_callback
    ]
)  # accuracy - training accuracy, val_accuracy - test (validation) accuracy

### Wyświetlenie statystyk treningu

In [ ]:
# strata
plt.plot(r.history['loss'], label='loss')
plt.plot(r.history['val_loss'], label='val_loss')
plt.legend()

In [ ]:
# dokładność
plt.plot(r.history['accuracy'], label='acc')
plt.plot(r.history['val_accuracy'], label='val_acc')
plt.legend()

### Macierz błędu (ang. confusion matrix)

In [ ]:
!pip install scikit-learn

In [ ]:
# Popatrzmy jeszcze na macierz błędu i zobaczmy z jakimi przykładami
# sieć sobie nie poradziła.
from sklearn.metrics import confusion_matrix
import numpy as np
import itertools

def plot_confusion_matrix(
    cm,
    classes,
    normalize=False,
    title='Confusion matrix',
    cmap=plt.cm.Blues
):
  """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`
  """

  if normalize:
    cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    print("Normalized confusion matrix")
  else:
    print("Confusion matrix, without normalization")

  print(cm)

  plt.imshow(cm, interpolation='nearest', cmap=cmap)
  plt.title(title)
  plt.colorbar()
  tick_marks = np.arange(len(classes))
  plt.xticks(tick_marks, classes, rotation=45)
  plt.yticks(tick_marks, classes)

  fmt = '.2f' if normalize else 'd'
  thresh = cm.max() / 2.
  for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
    plt.text(j, i, format(cm[i, j], fmt), horizontalalignment="center", color="white" if cm[i, j] > thresh else "black")

  plt.tight_layout()
  plt.ylabel('True label')
  plt.xlabel('Predicted label')
  plt.show()

p_test = model.predict(x_test).argmax(axis=1)
cm = confusion_matrix(y_test, p_test)
plot_confusion_matrix(cm, list(range(10)))

# 9 <--> 4, 9 <--> 7, 2 <--> 7, etc.

In [ ]:
# Wyświetlmy kilka nieprawidłowo zaklasyfikowanych obrazów
misclassified_idx = np.where(p_test != y_test)[0]
i = np.random.choice(misclassified_idx)
plt.imshow(x_test[i], cmap='gray')
plt.title(f"True label: {y_test[i]} Predicted: {p_test[i]}")

In [ ]:
%tensorboard --logdir './{log_dir}'